In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb
pd.set_option('max_rows',100)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
fixed=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/fixed_cost.csv')
dprice=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/demand_price.csv')
pdcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/production_cost.csv')
pdcap=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/production_capacity.csv')
dcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/delivery_cost.csv')
chdays=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/changeover_days.csv')
chcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/changeover_cost.csv')
forecast=pd.read_csv('/Users/shashank/Downloads/demand_forecast.csv')

In [13]:
pdcap[pdcap.Product_ID=='P13']

,Plant,Line,Product_ID,Capacity
12,A,1,P13,125
93,A,2,P13,140
174,B,1,P13,0
255,B,2,P13,0
336,B,3,P13,0
417,C,1,P13,181


In [3]:
seq=pd.read_csv('/Users/shashank/Downloads/manufacture_sequence-1.csv')

In [6]:
pdcap.rename(columns={'Product':'Product_ID'},inplace=True)

In [7]:
## seq with capacities
seq=seq.merge(pdcap,on=['Plant','Line','Product_ID'],how='left')

In [315]:
productions=seq.groupby(['Plant','Month','Product_ID']).sum()['Capacity']
productions=productions.reset_index(drop=False,)

In [316]:
forecast=forecast.merge(dprice,on=['Product_ID','Region'],)

In [371]:
one=forecast ## trial run for P16 for algo checking

In [448]:
one13=one[one.Product_ID=='P13']
one13.reset_index(drop=True,inplace=True)

In [449]:
da=dcost[dcost.Plant=='A']
da.reset_index(inplace=True,drop=True)
da.drop(['Plant'],inplace=True,axis=1)
da=dcost[dcost.Plant=='A']
da.reset_index(inplace=True,drop=True)
da.drop(['Plant'],inplace=True,axis=1)
one13=one13.merge(da,on='Region')
one13.rename(columns={'Delivery_cost':'ProfitA'},inplace=True)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
dcost.head()

,Plant,Region,Delivery_cost
0,A,R1,85.541475
1,A,R2,223.934332
2,A,R3,34.648618
3,A,R4,32.059332
4,A,R5,121.255760


In [450]:
da=dcost[dcost.Plant=='B']
da.reset_index(inplace=True,drop=True)
da.drop(['Plant'],inplace=True,axis=1)
one13=one13.merge(da,on='Region')
one13.rename(columns={'Delivery_cost':'ProfitB'},inplace=True)

da=dcost[dcost.Plant=='C']
da.reset_index(inplace=True,drop=True)
da.drop(['Plant'],inplace=True,axis=1)
one13=one13.merge(da,on='Region')
one13.rename(columns={'Delivery_cost':'ProfitC'},inplace=True)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [451]:
pdcap=pdcap.rename({'Product':'Product_ID'})

In [452]:
cap13=pdcap[pdcap.Product_ID=='P13']
cap13.reset_index(drop=True,inplace=True)
cap13A=cap13[cap13.Plant=='A']
Acan=int(cap13A.Capacity.sum())

cap13=pdcap[pdcap.Product_ID=='P13']
cap13.reset_index(drop=True,inplace=True)
cap13B=cap13[cap13.Plant=='B']
Bcan=int(cap13B.Capacity.sum())

cap13=pdcap[pdcap.Product_ID=='P13']
cap13.reset_index(drop=True,inplace=True)
cap13C=cap13[cap13.Plant=='C']
Ccan=int(cap13C.Capacity.sum())

print Acan,Bcan,Ccan

265 0 181


In [379]:
# one13['Acan']=Acan
# one13['Bcan']=Bcan
# one13['Ccan']=Ccan


In [329]:
# one13.drop(['Acan','Bcan','Ccan'],inplace=True,axis=1)

In [453]:
one13['RevA']=one13.Demand*one13.ProfitA
one13['RevB']=one13.Demand*one13.ProfitB
one13['RevC']=one13.Demand*one13.ProfitC

In [9]:
seq.head()

,Plant,Line,Month,Day,Product_ID,Capacity
0,A,1,37,1,P72,70.0
1,A,1,37,2,P72,70.0
2,A,1,37,3,P72,70.0
3,A,1,37,4,P72,70.0
4,A,1,37,5,P72,70.0


In [459]:
one13['Acan']=seq[seq.Month==37][seq.Plant=='A'][seq.Product_ID=='P13']['Capacity'].sum()
one13['Bcan']=seq[seq.Month==37][seq.Plant=='B'][seq.Product_ID=='P13']['Capacity'].sum()
one13['Ccan']=seq[seq.Month==37][seq.Plant=='C'][seq.Product_ID=='P13']['Capacity'].sum()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [462]:
one13=one13.sort_values(['RevA','RevB','RevC'],ascending=False)
one13.reset_index(inplace=True,drop=True)
one13.head()

,Product_ID,Region,Month,Demand,Demand_price,ProfitA,ProfitB,ProfitC,RevA,RevB,RevC,Acan,Bcan,Ccan
0,P13,R6,38,297.668149,1864.405246,34.648618,59.0,277.030075,10313.789836,17562.420785,82463.029615,420.0,0,0
1,P13,R6,37,238.116867,1864.405246,34.648618,59.0,277.030075,8250.420251,14048.895159,65965.533551,420.0,0,0
2,P13,R6,39,237.238869,1864.405246,34.648618,59.0,277.030075,8219.998827,13997.093265,65722.301644,420.0,0,0
3,P13,R8,38,65.461630,1593.444302,114.112903,242.0,170.235257,7470.016655,15841.714477,11143.877451,420.0,0,0
4,P13,R8,39,61.012262,1593.444302,114.112903,242.0,170.235257,6962.286319,14764.967344,10386.438090,420.0,0,0


In [10]:
pdcap

,Plant,Line,Product_ID,Capacity
0,A,1,P1,0
1,A,1,P2,0
2,A,1,P3,210
3,A,1,P4,0
4,A,1,P5,210
5,A,1,P6,0
6,A,1,P7,165
7,A,1,P8,195
8,A,1,P9,200
9,A,1,P10,0


In [413]:
one13=one13[one13.Month==37]

In [425]:
one13.reset_index(drop=True,inplace=True)

In [426]:
re=[]
a=one13.loc[0,'Acan']
b=one13.loc[0,'Bcan']
c=one13.loc[0,'Ccan']
d=one13.Demand.sum()
for ind,row in one13.iterrows():
    dic={}
    dic[row['RevA']]='A'
    dic[row['RevB']]='B'
    dic[row['RevC']]='C'
    if a<=0 and b<=0 and c<=0:
        break
    dem=row['Demand']
    l=[row['RevA'],row['RevB'],row['RevC']]
    maxR=np.max(l)
    plant=dic[maxR]
    if plant=='A':
        if a>=dem:
            a=a-dem
            re.append([s,plant,row['Region'],dem])
            dem=0
        else:
            dem=dem-a
            re.append([s,plant,row['Region'],a])
            a=0
            l=[row['RevB'],row['RevC']]
            maxR=np.max(l)
            plant=dic[maxR]
            if plant=='B':
                if b>=dem:
                    b=b-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-b
                    re.append([s,plant,row['Region'],b])
                    b=0
                    plant='C'
                    if c>=dem:
                        c=c-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-c
                        re.append([s,plant,row['Region'],c])
                        c=0
            elif plant=='C':
                if c>=dem:
                    c=c-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-c
                    re.append([s,plant,row['Region'],c])
                    c=0
                    plant='B'
                    if b>=dem:
                        b=b-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-b
                        re.append([s,plant,row['Region'],b])
                        b=0
                        
    elif plant=='B':
        if b>=dem:
            
            b=b-dem
            re.append([s,plant,row['Region'],dem])
            dem=0
        else:
            dem=dem-b
            re.append([s,plant,row['Region'],b])
            b=0
            l=[row['RevA'],row['RevC']]
            maxR=np.max(l)
            plant=dic[maxR]
            
            if plant=='A':
                if a>=dem:
                    
                    a=a-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-a
                    re.append([s,plant,row['Region'],a])
                    a=0
                    plant='C'
                    if c>=dem:
                        c=c-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-c
                        re.append([s,plant,row['Region'],c])
                        c=0
            elif plant=='C':
                
                if c>=dem:
                    c=c-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-c
                    re.append([s,plant,row['Region'],c])
                    c=0
                    plant='A'
                    if a>=dem:
                        
                        a=a-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-a
                        re.append([s,plant,row['Region'],a])
                        a=0
                
                        
    elif plant=='C':
        if c>=dem:
            c=c-dem
            re.append([s,plant,row['Region'],dem])
            dem=0
        else:
            dem=dem-c
            re.append([s,plant,row['Region'],c])
            c=0
            l=[row['RevB'],row['RevA']]
            maxR=np.max(l)
            plant=dic[maxR]
            
            if plant=='B':
                if b>=dem:
                    b=b-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-b
                    re.append([s,plant,row['Region'],b])
                    b=0
                    plant='A'
                    if a>=dem:
                        a=a-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-a
                        re.append([s,plant,row['Region'],dem])
                        a=0
            elif plant=='A':
                if a>=dem:
                    a=a-dem
                    re.append([s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-a
                    re.append([s,plant,row['Region'],a])
                    a=0
                    plant='B'
                    if b>=dem:
                        b=b-dem
                        re.append([s,plant,row['Region'],dem])
                        dem=0
                    else:
                        dem=dem-b
                        re.append([s,plant,row['Region'],b])
                        b=0

In [427]:
pd.DataFrame(re)

,0,1,2,3
0,P81,C,R6,154.586383
1,P81,B,R10,0.000000
2,P81,A,R10,0.000000
3,P81,C,R10,11.409470
4,P81,C,R4,11.706774
5,P81,C,R1,0.000000
6,P81,C,R11,0.000000
7,P81,C,R12,0.000000
8,P81,C,R13,0.000000
9,P81,C,R14,0.000000


In [428]:
a=pd.DataFrame(re)
a[3].sum()

177.70262757

In [167]:
df=pd.DataFrame(re)
df=df.rename(columns={0:'From',1:'To',2:'Quant'})

## Final algo for transportation
## Sorted the revenue ( demand price - transport cost from every region ) for each Plant to fulfill the demand from maximum revenue regions greedily.
## see the below dataframe.

In [461]:
one13[one13.Month==37].head()

,Product_ID,Region,Month,Demand,Demand_price,ProfitA,ProfitB,ProfitC,RevA,RevB,RevC,Acan,Bcan,Ccan
1,P13,R6,37,238.116867,1864.405246,34.648618,59.0,277.030075,8250.420251,14048.895159,65965.533551,420.0,0,0
8,P13,R8,37,34.425442,1593.444302,114.112903,242.0,170.235257,3928.387123,8330.956947,5860.423972,420.0,0,0
10,P13,R10,37,29.874036,1639.245650,112.327189,113.0,44.999574,3355.666503,3375.766086,1344.318914,420.0,0,0
11,P13,R11,37,19.946352,1639.349139,148.041475,233.0,213.722138,2952.887408,4647.500084,4262.977055,420.0,0,0
15,P13,R18,37,7.547520,1911.349098,107.862903,117.0,273.880474,814.097383,883.059800,2067.118259,420.0,0,0


In [429]:
## Transport to regions for different months seperately
one=forecast
res=pd.DataFrame()
for j in [37,38,39]:
    for i in range(1,82):
        s='P'+str(i)
        one13=one[one.Product_ID==s]
        one13.reset_index(drop=True,inplace=True)
        da=dcost[dcost.Plant=='A']
        da.reset_index(inplace=True,drop=True)
        da.drop(['Plant'],inplace=True,axis=1)
        da=dcost[dcost.Plant=='A']
        da.reset_index(inplace=True,drop=True)
        da.drop(['Plant'],inplace=True,axis=1)
        one13=one13.merge(da,on='Region')
        one13.rename(columns={'Delivery_cost':'ProfitA'},inplace=True)
        da=dcost[dcost.Plant=='B']
        da.reset_index(inplace=True,drop=True)
        da.drop(['Plant'],inplace=True,axis=1)
        one13=one13.merge(da,on='Region')
        one13.rename(columns={'Delivery_cost':'ProfitB'},inplace=True)

        da=dcost[dcost.Plant=='C']
        da.reset_index(inplace=True,drop=True)
        da.drop(['Plant'],inplace=True,axis=1)
        one13=one13.merge(da,on='Region')
        one13.rename(columns={'Delivery_cost':'ProfitC'},inplace=True)
        cap13=pdcap[pdcap.Product_ID==s]
        cap13.reset_index(drop=True,inplace=True)
        cap13A=cap13[cap13.Plant=='A']
        Acan=int(cap13A.Capacity.sum())

        cap13=pdcap[pdcap.Product_ID==s]
        cap13.reset_index(drop=True,inplace=True)
        cap13B=cap13[cap13.Plant=='B']
        Bcan=int(cap13B.Capacity.sum())

        cap13=pdcap[pdcap.Product_ID==s]
        cap13.reset_index(drop=True,inplace=True)
        cap13C=cap13[cap13.Plant=='C']
        Ccan=int(cap13C.Capacity.sum())

        one13['Acan']=seq[seq.Month==j][seq.Plant=='A'][seq.Product_ID==s]['Capacity'].sum()
        one13['Bcan']=seq[seq.Month==j][seq.Plant=='B'][seq.Product_ID==s]['Capacity'].sum()
        one13['Ccan']=seq[seq.Month==j][seq.Plant=='C'][seq.Product_ID==s]['Capacity'].sum()
        one13['RevA']=one13.Demand*one13.ProfitA
        one13['RevB']=one13.Demand*one13.ProfitB
        one13['RevC']=one13.Demand*one13.ProfitC
        one13=one13.sort_values(['RevA','RevB','RevC'],ascending=False)
        one13.reset_index(inplace=True,drop=True)
        one13=one13[one13.Month==j]
        one13=one13.reset_index(drop=True)
        re=[]
        a=one13.loc[0,'Acan']
        b=one13.loc[0,'Bcan']
        c=one13.loc[0,'Ccan']
        d=one13.Demand.sum()
        for ind,row in one13.iterrows():
            dic={}
            dic[row['RevA']]='A'
            dic[row['RevB']]='B'
            dic[row['RevC']]='C'
            if a<=0 and b<=0 and c<=0:
                break
            dem=row['Demand']
            l=[row['RevA'],row['RevB'],row['RevC']]
            maxR=np.max(l)
            plant=dic[maxR]
            if plant=='A':
                if a>=dem:
                    a=a-dem
                    re.append([row['Month'],s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-a
                    if a>0:
                        re.append([row['Month'],s,plant,row['Region'],a])
                    a=0
                    l=[row['RevB'],row['RevC']]
                    maxR=np.max(l)
                    plant=dic[maxR]
                    if plant=='B':
                        if b>=dem:
                            b=b-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-b
                            if b>0:
                                re.append([row['Month'],s,plant,row['Region'],b])
                            b=0
                            plant='C'
                            if c>=dem:
                                c=c-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-c
                                if c>0:
                                    re.append([row['Month'],s,plant,row['Region'],c])
                                c=0
                    elif plant=='C':
                        if c>=dem:
                            c=c-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-c
                            if c>0:
                                re.append([row['Month'],s,plant,row['Region'],c])
                            c=0
                            plant='B'
                            if b>=dem:
                                b=b-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-b
                                if b>0:
                                    re.append([row['Month'],s,plant,row['Region'],b])
                                b=0

            elif plant=='B':
                if b>=dem:

                    b=b-dem
                    re.append([row['Month'],s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-b
                    if b>0:
                        re.append([row['Month'],s,plant,row['Region'],b])
                    b=0
                    l=[row['RevA'],row['RevC']]
                    maxR=np.max(l)
                    plant=dic[maxR]

                    if plant=='A':
                        if a>=dem:

                            a=a-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-a
                            if a>0:
                                re.append([row['Month'],s,plant,row['Region'],a])
                            a=0
                            plant='C'
                            if c>=dem:
                                c=c-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-c
                                if c>0:
                                    re.append([row['Month'],s,plant,row['Region'],c])
                                c=0
                    elif plant=='C':

                        if c>=dem:
                            c=c-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-c
                            if c>0:
                                re.append([row['Month'],s,plant,row['Region'],c])
                            c=0
                            plant='A'
                            if a>=dem:

                                a=a-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-a
                                if a>0:
                                    re.append([row['Month'],s,plant,row['Region'],a])
                                a=0


            elif plant=='C':
                if c>=dem:
                    c=c-dem
                    re.append([row['Month'],s,plant,row['Region'],dem])
                    dem=0
                else:
                    dem=dem-c
                    if c>0:
                        re.append([row['Month'],s,plant,row['Region'],c])
                    c=0
                    l=[row['RevB'],row['RevA']]
                    maxR=np.max(l)
                    plant=dic[maxR]

                    if plant=='B':
                        if b>=dem:
                            b=b-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-b
                            if b>0:
                                re.append([row['Month'],s,plant,row['Region'],b])
                            b=0
                            plant='A'
                            if a>=dem:
                                a=a-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-a
                                if a>0:
                                    re.append([row['Month'],s,plant,row['Region'],a])
                                a=0
                    elif plant=='A':
                        if a>=dem:
                            a=a-dem
                            re.append([row['Month'],s,plant,row['Region'],dem])
                            dem=0
                        else:
                            dem=dem-a
                            if a>0:
                                re.append([row['Month'],s,plant,row['Region'],a])
                            a=0
                            plant='B'
                            if b>=dem:
                                b=b-dem
                                re.append([row['Month'],s,plant,row['Region'],dem])
                                dem=0
                            else:
                                dem=dem-b
                                if b>0:
                                    re.append([row['Month'],s,plant,row['Region'],b])
                                b=0
        temp=pd.DataFrame(re)
        res=pd.concat([res,temp])
    

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A

In [430]:
res

,0,1,2,3,4
0,37,P3,A,R8,228.627164
1,37,P3,A,R18,150.301366
2,37,P3,A,R7,60.792864
3,37,P3,A,R5,29.310552
4,37,P3,A,R11,9.805430
5,37,P3,C,R1,0.000000
6,37,P3,C,R10,0.000000
7,37,P3,C,R12,0.000000
8,37,P3,C,R13,0.000000
9,37,P3,C,R14,0.000000


In [431]:
res=res.rename(columns={0:'Month',1:'Product_ID',2:'Plant',3:'Region',4:'Shipping_to_region_quantity'})

In [433]:
res.reset_index(drop=True,inplace=True)

In [463]:
res[res.Month==37][res.Product_ID=='P13'] ## Transport greedily

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Month,Product_ID,Plant,Region,Shipping_to_region_quantity
42,37,P13,A,R6,238.116867
43,37,P13,A,R8,34.425442
44,37,P13,A,R10,29.874036
45,37,P13,A,R11,19.946352
46,37,P13,A,R18,7.547520
47,37,P13,A,R12,16.752675
48,37,P13,A,R3,15.933166
49,37,P13,C,R1,0.000000
50,37,P13,C,R13,0.000000
51,37,P13,C,R14,0.000000


In [435]:
res.reset_index(drop=True,inplace=True)

In [441]:
t=pd.read_csv('/Users/shashank/Downloads/av_or/sample_submission_BqRDhl5/shipping_region.csv')

In [ ]:
## Merge for submission

In [442]:
data=t.merge(res,on=['Plant','Region','Product_ID','Month'],how='left')

In [443]:
data.drop(['Shipping_to_region_quantity_x'],inplace=True,axis=1)

In [444]:
data.fillna(0,inplace=True)

In [445]:
data=data.rename(columns={'Shipping_to_region_quantity_y':'Shipping_to_region_quantity'})

In [446]:
data.Shipping_to_region_quantity.sum() ## total transported

46176.35585746696

In [251]:
data['Shipping_to_region_quantity']=np.floor(data.Shipping_to_region_quantity)

In [447]:
data.to_csv('/Users/shashank/Desktop/lastday/shipping_region.csv',index=False)

In [243]:
forecast.to_csv('/Users/shashank/Desktop/lastday/demand_forecast.csv',index=False)

In [247]:
seq.to_csv('/Users/shashank/Desktop/lastday/manufacture_sequence.csv',index=False)